In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
from sqlalchemy import create_engine
import sys
import sqlite3
import os

In [185]:
# reading in a film list exported from letterboxd
df = pd.read_csv('download/data/ratings.csv')
df.head()

In [ ]:
# environment variables
API_HOST = os.getenv('RAPID_HOST')
API_KEY = os.getenv('RAPID_KEY')
API_URL = os.getenv('RAPID_URL')

In [186]:
### FUNCTIONS ###

# function to get url from index position in dataframe
def get_lurl(i):
    url = df.iloc[i][3]
    return url

# function to get imdb tt from letterboxd url
def get_tt(u):
    page = requests.get(u)
    soup = BeautifulSoup(page.content, 'html.parser')
    links = []
    for link in soup.body.find_all('a'):
        links.append(link.get('href'))
    tt = ''
    for link in links:
        if link != None:
            if 'imdb.com/title/tt' in link:
                tt += link.split('/')[4]
        if len(tt) == 9:
            return tt

# function that takes an index position, connects to rapidapi, and returns the rating
def get_more(i):
    tt = get_tt(get_lurl(i))
    url = API_URL
    querystring = querystring = {"i":tt,"r":"json"}
    # TODO REMOVE THESE KEYS
    headers = {
    'x-rapidapi-key': API_KEY,
    'x-rapidapi-host': API_HOST
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    return json.loads(response.text)

In [ ]:
# adding additional film details from rapidapi
df["More"] = ""
for i in range(0, len(df)):
    df["More"][i] = get_more(i)

In [456]:
### SAVING THE RETRIEVED DETAILS because that took a while ###
saved = df.copy()

In [ ]:
# adding blank columns
df["Rated"] = ''
df["Runtime"] = ''
df["Genre"] = ''
df["IMDb"] = ''
df["Rotten Tomatoes"] = ''
df["Metacritic"] = ''
df["BoxOffice"] = ''

# extracting and formatting info retrieved from api
for i in range(0, len(df)):
    if "Rated" in df.More[i]:
        df.Rated[i] = df.More[i]["Rated"]
    else:
        df.Rated[i] = "Unrated"

for i in range(0, len(df)):
    if "Runtime" in df.More[i]:
        df.Runtime[i] = df.More[i]["Runtime"]

for i in range(0, len(df)):
    if "Genre" in df.More[i]:
        df.Genre[i] = df.More[i]["Genre"].split(',')[0]

for i in range(0, len(df)):
    if "imdbRating" in df.More[i]:
        df.IMDb[i] = df.More[i]["imdbRating"]

for i in range(0, len(df)):
    if "Ratings" in df.More[i]:
        for x in df.More[i]["Ratings"]:
            if 'Rotten' in x['Source']:
                df["Rotten Tomatoes"][i] = x["Value"]

for i in range(0, len(df)):
    if "Ratings" in df.More[i]:
        for x in df.More[i]["Ratings"]:
            if 'Meta' in x['Source']:
                df.Metacritic[i] = x["Value"]

for i in range(0, len(df)):
    if "BoxOffice" in df.More[i]:
        df.BoxOffice[i] = df.More[i]["BoxOffice"]

In [470]:
df.head()

,Name,Year,Rating,Rated,Genre,IMDb,Rotten Tomatoes,Metacritic,BoxOffice,Runtime,More
0,Star Wars,1977,4.0,PG,Action,8.6,92%,90/100,"$460,998,507",121 min,{'Title': 'Star Wars: Episode IV - A New Hope'...
1,Breaking Away,1979,4.0,PG,Comedy,7.7,95%,91/100,"$16,424,918",101 min,"{'Title': 'Breaking Away', 'Year': '1979', 'Ra..."
2,The Sound of Music,1965,3.5,G,Biography,8.0,83%,63/100,"$159,287,539",172 min,"{'Title': 'The Sound of Music', 'Year': '1965'..."
3,One Hundred and One Dalmatians,1961,4.0,G,Animation,7.3,98%,83/100,"$144,880,014",79 min,"{'Title': 'One Hundred and One Dalmatians', 'Y..."
4,Robin Hood,1973,3.5,G,Animation,7.6,54%,57/100,N/A,83 min,"{'Title': 'Robin Hood', 'Year': '1973', 'Rated..."


In [485]:
con = sqlite3.connect('movie_details.db')

In [529]:
# now put it in a database
con = sqlite3.connect('movie_details.db')
df.to_sql('movie_details', con)

C:\Users\Collin\.virtualenvs\fucking_with_film-31rp7xRM\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [2]:
test = pd.DataFrame()

In [6]:
test['1'] = ''